In [1]:
# !pip install -U langchain-cli

langchain-cli                            0.0.21


Create the app

In [3]:
!yes|langchain app new chat-with-pdf --package rag-chroma

Would you like to install these templates into your environment with pip? [y/N]: Adding https://github.com/langchain-ai/langchain.git@master...
 - Downloaded templates/rag-chroma to rag-chroma
Running: pip install -e \
  chat-with-pdf/packages/rag-chroma
Obtaining file:///Users/greatmaster/Desktop/projects/oreilly-live-trainings/oreilly_live_training_agents/notebooks/chat-with-pdf/packages/rag-chroma
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for rag-chroma (pyproject.toml) ... done
  Created wheel for rag-chroma: filename=rag_chroma-0.1.0-py3-none-any.whl size=2903 sha256=145eb68c06d281f0a69c4fe3ba15eb20e4ab04e92af14667149aee98e52fa074
  Stored in directory: /private/var/folders/5l/y8s3fc655417629rqwgxkhx80000gn/T/pip-ephem-wheel-cache-1xd9tszb/wheels/ab/82/84/353726914d2db0738fdfea77afdc6c7d1b03b64399b

Find the `/app/server.py` file and add:

```
from rag_conversation import chain as rag_chroma_chain

add_routes(app, rag_chroma_chain, path="/rag-chroma")
```

Optionally you can configure langsmith:

```
export LANGCHAIN_TRACING_V2=true
export LANGCHAIN_API_KEY=<your-api-key>
export LANGCHAIN_PROJECT=<your-project>  # if not specified, defaults to "default"
```

Now, let's add some data to chat with!

For that let's open the chain.py file that was created in `/packages/rag-chroma/rag-chroma/chain.py`

Let's modify it to accept data releavnt to us, in our case let's set up a chat with a blog article about agents:

- https://lilianweng.github.io/posts/2023-06-23-agent/

```
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

# Add to vectorDB
vectorstore = Chroma.from_documents(documents=all_splits, 
                                    collection_name="rag-chroma",
                                    embedding=OpenAIEmbeddings(),
                                    )
retriever = vectorstore.as_retriever()
```

The rest of the code should remain the same.

Now, we can deploy our app:

In [6]:
! cd chat-with-pdf && langchain serve

INFO:     Will watch for changes in these directories: ['/Users/greatmaster/Desktop/projects/oreilly-live-trainings/oreilly_live_training_agents/notebooks/chat-with-pdf']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [4574] using WatchFiles
/Users/greatmaster/miniconda3/envs/oreilly-langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/Users/greatmaster/miniconda3/envs/oreilly-langchain/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_com

Let's access our chat by going to:

http://localhost:8000/rag-chroma/playground

To access the the template from code run this:
(make sure to serve the app in a terminal first with the command shown previously).

In [11]:
from langserve.client import RemoteRunnable

runnable = RemoteRunnable("http://localhost:8000/rag-chroma")

In [12]:
runnable

In [13]:
runnable.invoke("How does tool use work in agents?")

'Tool use in agents works by equipping the agents with external tools, which significantly extends their capabilities. These tools allow the agents to browse the internet, read documentation, execute code, call robotics experimentation APIs, and leverage other large language models (LLMs) for additional information that may be missing from the model weights. Tool use enables the agents to go beyond their physical and cognitive limits and perform tasks that require external resources and expertise.'